```{contents}
```
## **Arbitration in LangGraph**

**Arbitration** in LangGraph is a **control and decision mechanism** used in **multi-agent systems** to resolve conflicts, select among competing proposals, and determine the next execution path of the graph. It formalizes *who decides what happens next* when multiple agents, tools, or strategies produce different outputs.

---

### **1. Why Arbitration Is Needed**

In real-world LangGraph systems, multiple agents often produce:

* Conflicting answers
* Different plans
* Competing tool choices
* Inconsistent confidence levels

Without arbitration, the system becomes unstable and unreliable.

Arbitration provides:

| Capability     | Purpose                      |
| -------------- | ---------------------------- |
| Consistency    | One authoritative decision   |
| Safety         | Prevents dangerous actions   |
| Optimality     | Chooses best solution        |
| Governance     | Enforces policies            |
| Explainability | Logs why a decision was made |

---

### **2. Conceptual Model**

```
Agents / Tools / Subgraphs
        ↓
   Arbitration Node
        ↓
 Final Decision / Next Node
```

Arbitration acts as a **decision authority** over parallel or competing outputs.

---

### **3. Forms of Arbitration**

| Type         | Description                         |
| ------------ | ----------------------------------- |
| Rule-Based   | Deterministic selection using rules |
| Score-Based  | Choose highest scoring candidate    |
| LLM-Based    | LLM judges outputs                  |
| Hybrid       | Rules + LLM                         |
| Human-Guided | Requires human approval             |

---

### **4. Where Arbitration Appears in LangGraph**

* Multi-agent coordination
* Planner–Executor systems
* Debate & consensus workflows
* Tool selection
* Safety gating
* Policy enforcement

---

### **5. Arbitration as a Node**

Arbitration is implemented as a **node** that:

1. Receives multiple candidate results
2. Evaluates them
3. Selects or merges them
4. Updates the shared state

---

### **6. State Design for Arbitration**

```python
class State(TypedDict):
    proposals: list
    decision: str
    confidence: float
```

---

### **7. Simple Rule-Based Arbitration Example**

```python
def arbitration_node(state):
    best = max(state["proposals"], key=lambda x: x["score"])
    return {"decision": best["answer"], "confidence": best["score"]}
```

---

### **8. LLM-Based Arbitration Example**

```python
def llm_arbitrator(state):
    prompt = f"""
    Choose the best answer based on accuracy, safety, and clarity:

    {state['proposals']}
    """
    decision = llm.invoke(prompt)
    return {"decision": decision}
```

---

### **9. Arbitration in a Multi-Agent Graph**

```
Research Agent  ─┐
Analysis Agent  ├─→ Arbitration Node → Final Answer
Safety Agent    ─┘
```

```python
builder.add_node("arbitrate", arbitration_node)
builder.add_edge("agent1", "arbitrate")
builder.add_edge("agent2", "arbitrate")
builder.add_edge("agent3", "arbitrate")
```

---

### **10. Advanced Arbitration Patterns**

| Pattern                  | Purpose                   |
| ------------------------ | ------------------------- |
| Winner-Takes-All         | Select single best        |
| Weighted Voting          | Weighted agent trust      |
| Safety Override          | Block unsafe actions      |
| Confidence Gating        | Require minimum certainty |
| Hierarchical Arbitration | Supervisor agent          |

---

### **11. Arbitration vs Consensus**

| Arbitration      | Consensus            |
| ---------------- | -------------------- |
| Single authority | Collective agreement |
| Fast             | Slower               |
| Deterministic    | Negotiated           |
| Centralized      | Distributed          |

---

### **12. Production Considerations**

| Requirement    | Implementation       |
| -------------- | -------------------- |
| Traceability   | Log every decision   |
| Explainability | Store rationale      |
| Bias Control   | Calibrate arbitrator |
| Fail-Safe      | Human override       |
| Scalability    | Efficient evaluation |

---

### **13. Why Arbitration Is Critical**

Without arbitration:

* Agents conflict endlessly
* Decisions become unstable
* Systems behave unpredictably

With arbitration:

> **LangGraph becomes a reliable decision engine instead of a fragile pipeline.**


### Demonstration

In [1]:
# --- Single Cell Demonstration: Multi-Agent Arbitration in LangGraph ---

from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Shared State Definition
# ---------------------------
class State(TypedDict):
    question: str
    proposals: List[dict]
    decision: str

# ---------------------------
# 2. Three Competing Agents
# ---------------------------
def agent_a(state: State):
    proposal = {"agent": "A", "answer": "Paris", "score": 0.85}
    return {"proposals": state["proposals"] + [proposal]}

def agent_b(state: State):
    proposal = {"agent": "B", "answer": "Paris, France", "score": 0.90}
    return {"proposals": state["proposals"] + [proposal]}

def agent_c(state: State):
    proposal = {"agent": "C", "answer": "Lyon", "score": 0.40}
    return {"proposals": state["proposals"] + [proposal]}

# ---------------------------
# 3. Arbitration Node
# ---------------------------
def arbitration_node(state: State):
    best = max(state["proposals"], key=lambda x: x["score"])
    return {"decision": best["answer"]}

# ---------------------------
# 4. Graph Construction
# ---------------------------
builder = StateGraph(State)

builder.add_node("agent_a", agent_a)
builder.add_node("agent_b", agent_b)
builder.add_node("agent_c", agent_c)
builder.add_node("arbitrate", arbitration_node)

builder.set_entry_point("agent_a")
builder.add_edge("agent_a", "agent_b")
builder.add_edge("agent_b", "agent_c")
builder.add_edge("agent_c", "arbitrate")
builder.add_edge("arbitrate", END)

graph = builder.compile()

# ---------------------------
# 5. Execute
# ---------------------------
initial_state = {"question": "What is the capital of France?", "proposals": []}
result = graph.invoke(initial_state)

# ---------------------------
# 6. Output
# ---------------------------
print("All Proposals:")
for p in result["proposals"]:
    print(p)

print("\nFinal Decision:", result["decision"])


All Proposals:
{'agent': 'A', 'answer': 'Paris', 'score': 0.85}
{'agent': 'B', 'answer': 'Paris, France', 'score': 0.9}
{'agent': 'C', 'answer': 'Lyon', 'score': 0.4}

Final Decision: Paris, France
